### Importing Libs

In [2]:
import string

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import (
    GRU,
    LSTM,
    Bidirectional,
    Concatenate,
    Dense,
    Dot,
    Dropout,
    Embedding,
    Input,
    Lambda,
    Layer,
    Masking,
    SimpleRNN,
    Softmax,
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import plot_model

2023-05-19 10:03:37.162378: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-19 10:03:37.490528: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-19 10:03:37.492329: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 10:03:39.291535: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Loading Data

In [3]:
train = pd.read_csv("./datasets/squad10k.csv")
train.head(3)

,question,context,text,c_id,start_word,end_word
0,What types of wisdom did Mackintosh say Burke ...,Burke's Reflections sparked a pamphlet war. Th...,political and moral,10068,127,129
1,How many monasteries were destroyed during the...,After the Dalai Lama's government fled to Dhar...,"6,000",12309,52,52
2,In what year did Time magazine choose Seattle ...,Seattle is home to the University of Washingto...,2001,7465,122,122


In [4]:
test = pd.read_csv("./datasets/squadtest1k.csv")
test.head(3)

,question,context,answer1,answer2,answer3,answer4
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,France,France,France,France
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,10th and 11th centuries,in the 10th and 11th centuries,10th and 11th centuries,10th and 11th centuries
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,"Denmark, Iceland and Norway","Denmark, Iceland and Norway","Denmark, Iceland and Norway","Denmark, Iceland and Norway"


### Preparing data

In [5]:
paragraphs = []
questions = []
embed = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")

for text in train.context:
    words = np.array(text_to_word_sequence(text))
    paragraphs.append(embed(tf.constant(words)))

for text in train.question:
    words = np.array(text_to_word_sequence(text))
    questions.append(embed(tf.constant(words)))

2023-05-19 10:03:42.000696: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.
2023-05-19 10:03:42.694488: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'sentences' with dtype string and shape [?]
	 [[{{node sentences}}]]


In [6]:
padded_paragraphs = pad_sequences(paragraphs, padding="post", dtype="float32")
padded_questions = pad_sequences(questions, padding="post", dtype="float32")

In [7]:
batch_size = np.shape(padded_paragraphs)[0]
max_paragraph_length = np.shape(padded_paragraphs)[1]
max_question_length = np.shape(padded_questions)[1]
embedding_dimension = np.shape(padded_paragraphs)[2]

print("Shape of the Padded Embedded Paragraphs: ", np.shape(padded_paragraphs))
print("Shape of the Padded Embedded Questions: ", np.shape(padded_questions))
print("i.e. (Batch Size, Sequence Length, Embed Dimension)")

y_start_word = np.array(train.start_word)
y_end_word = np.array(train.end_word)
print("Shape of the Y Train set for Start Word: ", np.shape(y_start_word))
print("Shape of the Y Train set for End Word: ", np.shape(y_end_word))

Shape of the Padded Embedded Paragraphs:  (10000, 498, 128)
Shape of the Padded Embedded Questions:  (10000, 31, 128)
i.e. (Batch Size, Sequence Length, Embed Dimension)
Shape of the Y Train set for Start Word:  (10000,)
Shape of the Y Train set for End Word:  (10000,)


In [ ]:
(
    paragraphs_train,
    paragraphs_val,
    questions_train,
    questions_val,
    answer_start_train,
    answer_start_val,
    answer_end_train,
    answer_end_val,
) = train_test_split(
    padded_paragraphs,
    padded_questions,
    y_start_word,
    y_end_word,
    test_size=0.1,
    random_state=30,
)

### Metrics and Eval

In [ ]:
def exact_match(predicted_answer, true_answer):
    truth = str(true_answer).replace("-", " ")
    truth = "".join(token for token in truth if token not in string.punctuation)
    return np.sum(str(predicted_answer).lower() == str(true_answer).lower())


def f1_score(predicted_answer, true_answer):
    predicted = text_to_word_sequence(str(predicted_answer))
    truth = text_to_word_sequence(str(true_answer))
    true_positive = [i for i in predicted if i in truth]
    if len(true_positive) == 0:
        f1 = 0
    else:
        precision = len(true_positive) / len(predicted)
        recall = len(true_positive) / len(truth)
        f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [1]:
paragraphs = Input(
    shape=(max_paragraph_length, embedding_dimension), name="paragraphs_input"
)
paragraph = Masking(mask_value=0)(paragraphs)
paragraph = GRU(
    256,
    return_sequences=True,
    name="paragraph_out",
    kernel_regularizer=regularizers.l2(0.002),
    kernel_initializer="glorot_normal",
)(paragraph)

NameError: name 'Input' is not defined

In [2]:
questions = Input(
    shape=(max_question_length, embedding_dimension), name="questions_input"
)
question = Masking(mask_value=0)(questions)
question = GRU(
    256,
    return_sequences=True,
    name="question_gru",
    kernel_regularizer=regularizers.l2(0.002),
    kernel_initializer="glorot_normal",
)(question)

NameError: name 'Input' is not defined

In [ ]:
weights = Dense(1, activation="softmax", name="weights")(question)
question = Dot(axes=1, name="question_out")([weights, question])

In [ ]:
question_start = Dense(
    256,
    activation="linear",
    name="s1",
    use_bias=False,
    kernel_regularizer=regularizers.l2(0.001),
)(question)
answer_start = Dot(axes=(2, 2), name="s2")([paragraph, question_start])
question_end = Dense(
    256,
    activation="linear",
    name="e1",
    use_bias=False,
    kernel_regularizer=regularizers.l2(0.001),
)(question)
answer_end = Dot(axes=(2, 2), name="e2")([paragraph, question_end])

model = Model(
    inputs=[paragraphs, questions], outputs=[answer_start, answer_end]
)
plot_model(model, to_file="Baseline.png", show_shapes=True)

In [ ]:
acc = tf.keras.metrics.SparseCategoricalAccuracy()
opt = tf.keras.optimizers.Adamax()
sce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(
    optimizer=opt, loss=[sce, sce], loss_weights=[1, 1], metrics=[[acc], [acc]]
)

In [ ]:
checkpoint = ModelCheckpoint(
    filepath="basemodel",
    frequency="epoch",
    save_weights_only=True,
    save_best_only=True,
    verbose=0,
)

In [ ]:
history = model.fit(
    [paragraphs_train, questions_train],
    [answer_start_train, answer_end_train],
    validation_data=(
        [paragraphs_val, questions_val],
        [answer_start_val, answer_end_val],
    ),
    epochs=1,
    batch_size=64,
    callbacks=[checkpoint],
    verbose=0,
)

In [ ]:
model.load_weights("basemodel")

In [ ]:
def create_masked_matrix(pred_start, pred_end):
    # Creating the masked matrix of possible answers (where start < end < start 15)
    masked_matrix = tf.matmul(pred_start, tf.transpose(pred_end, [0, 2, 1]))
    i, j = np.meshgrid(
        *map(np.arange, (masked_matrix.shape[1], masked_matrix.shape[2])),
        indexing="ij"
    )
    masked_matrix.mask = (i <= j) & (j < i + 15)
    masked_matrix = np.where(masked_matrix.mask, masked_matrix, 0)
    max_results = np.amax(masked_matrix, axis=(1, 2))
    return masked_matrix, max_results


def model_eval(pred):
    pred_start = tf.exp(pred[0])
    pred_end = tf.exp(pred[1])

    masked_matrix, max_results = create_masked_matrix(pred_start, pred_end)
    number_of_examples = masked_matrix.shape[0]

    em = []
    f1 = []

    # Find the most probable answer for each question in the test set.
    # We compare with the four human answers, and keep the max F1 and EM scores.
    for k in range(number_of_examples):
        result = np.where(masked_matrix[k] == max_results[k])
        if result[1][0] < len(text_to_word_sequence(test.context[k])):
            answer = np.array(text_to_word_sequence(test.context[k]))[
                result[0][0] : result[1][0] + 1
            ]
        else:
            answer = ["-"]

        if result[0][0] != result[1][0] and result[1][0] < len(
            text_to_word_sequence(test.context[k])
        ):
            answer = " ".join(answer)
        else:
            answer = str(answer[0])
        em_k = max(
            exact_match(answer, test.answer1[k]),
            exact_match(answer, test.answer2[k]),
            exact_match(answer, test.answer3[k]),
            exact_match(answer, test.answer4[k]),
        )
        f1_k = max(
            f1_score(answer, test.answer1[k]),
            f1_score(answer, test.answer2[k]),
            f1_score(answer, test.answer3[k]),
            f1_score(answer, test.answer4[k]),
        )
        em.append(em_k)
        f1.append(f1_k)

    print("Exact Match: ", np.round(np.mean(em), 3))
    print("F1 Score: ", np.round(np.mean(f1), 3))

    return (em, f1)

In [ ]:
paragraph_test = []
question_test = []
embed = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")  # NNLM

for text in test.context:
    words = np.array(text_to_word_sequence(text))
    paragraph_test.append(embed(tf.constant(words)))

for text in test.question:
    words = np.array(text_to_word_sequence(text))
    question_test.append(embed(tf.constant(words)))

p_test = pad_sequences(
    paragraph_test, padding="post", dtype="float32", maxlen=max_paragraph_length
)
q_test = pad_sequences(
    question_test, padding="post", dtype="float32", maxlen=max_question_length
)

In [ ]:
pred_test = model.predict([p_test, q_test])
print("**Results on Test Set:")
(em_model, f1_model) = model_eval(pred_test)

In [ ]:
paragraphs = Input(
    shape=(max_paragraph_length, embedding_dimension), name="par0"
)
p = Masking(mask_value=0)(paragraphs)

In [ ]:
p = Bidirectional(
    GRU(
        128,
        return_sequences=True,
        name="par1",
        kernel_initializer="glorot_normal",
    ),
    merge_mode="concat",
)(p)
p = Dropout(0.15)(p)

p = Bidirectional(
    GRU(
        64,
        return_sequences=True,
        name="par2",
        kernel_initializer="glorot_normal",
    ),
    merge_mode="concat",
)(p)
p = Dropout(0.15)(p)

In [ ]:
questions = Input(
    shape=(max_question_length, embedding_dimension), name="ques0"
)
q = Masking(mask_value=0)(questions)
q = GRU(256, return_sequences=True, name="ques2")(q)
q = Dropout(0.15)(q)

In [ ]:
weights = Dense(1, activation="softmax", name="weights")(q)
q = Dot(axes=1, name="ques3")([weights, q])

In [ ]:
question_start = Dense(
    128,
    activation="linear",
    name="s1",
    use_bias=False,
    kernel_regularizer=regularizers.l2(0.002),
)(q)
answer_start = Dot(axes=(2, 2), name="s2")([p, question_start])

question_end = Dense(
    128,
    activation="linear",
    name="e1",
    use_bias=False,
    kernel_regularizer=regularizers.l2(0.002),
)(q)
answer_end = Dot(axes=(2, 2), name="e2")([p, question_end])
# Output is = a probability vector (None, seq_pars, 1) for each

# Model
model = Model(
    inputs=[paragraphs, questions], outputs=[answer_start, answer_end]
)

In [ ]:
def create_masked_matrix_for_one(pred_start, pred_end):
    # Creating the masked matrix of possible answers (where start < end < start 15)
    masked_matrix = tf.matmul(pred_start, tf.transpose(pred_end))
    i, j = np.meshgrid(*map(np.arange, (masked_matrix.shape)), indexing="ij")
    masked_matrix.mask = (i <= j) & (j < i + 15)
    masked_matrix = np.where(masked_matrix.mask, masked_matrix, 0)
    max_results = np.where(masked_matrix == np.amax(masked_matrix))
    return masked_matrix, max_results


# Function to get the result on the kth question
def get_result(k, model=model, verbose=True):
    paragraph = tf.expand_dims(p_test[k], 0)
    question = tf.expand_dims(q_test[k], 0)
    out = model([paragraph, question])
    start = tf.exp(out[0][0])
    end = tf.exp(out[1][0])

    _, result = create_masked_matrix_for_one(start, end)

    if result[1][0] < len(text_to_word_sequence(test.context[k])):
        answer = np.array(text_to_word_sequence(test.context[k]))[
            result[0][0] : result[1][0] + 1
        ]
    else:
        answer = ["-"]

    if result[0][0] != result[1][0] and result[1][0] < len(
        text_to_word_sequence(test.context[k])
    ):
        answer = " ".join(answer)
    else:
        answer = str(answer[0])
    if verbose:
        print("--------------------------------------------------------")
        print("Question: ", test.question[k])
        print("--------------------------------------------------------")
        print("Context: ")
        print(test.context[k])
    print("--------------------------------------------------------")
    print("Model's answer: ", answer)
    print("Human answers: ")
    print(
        test.answer1[k],
        " -- ",
        test.answer2[k],
        " -- ",
        test.answer3[k],
        " -- ",
        test.answer4[k],
    )
    print("--------------------------------------------------------")
    print(
        "EM Score: ",
        max(
            exact_match(answer, test.answer1[k]),
            exact_match(answer, test.answer2[k]),
            exact_match(answer, test.answer3[k]),
            exact_match(answer, test.answer4[k]),
        ),
    )
    print(
        "F1 Score: ",
        np.round(
            max(
                f1_score(answer, test.answer1[k]),
                f1_score(answer, test.answer2[k]),
                f1_score(answer, test.answer3[k]),
                f1_score(answer, test.answer4[k]),
            ),
            3,
        ),
    )

In [ ]:
get_result(10)